In [92]:
import cv2
import torch
import numpy as np
import torch.nn as nn
from sklearn.cluster import KMeans

# from utils.torch_utils import prune
from utils.torch_utils import select_device
from models.experimental import attempt_load

from utils.plots import plot_one_box
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression
from utils.general import apply_classifier, scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path

In [93]:
def prune(model, amount=0.3):
    # Prune model to requested global sparsity
    import torch.nn.utils.prune as prune
    print('Pruning model... ', end='')
    for name, m in model.named_modules():
        if isinstance(m, nn.Conv2d):
            prune.l1_unstructured(m, name='weight', amount=amount)  # prune
            prune.remove(m, 'weight')  # make permanent
            
    return model

In [99]:
device = '0'
weights = './runs/train/yolov7-custom/weights/best.pt'

device = select_device(device)
model = attempt_load(weights, map_location=device)  # load FP32 model
# stride = int(model.stride.max())  # model stride

if device.type != 'cpu':
    model(torch.zeros(1, 3, 640, 640).to(device).type_as(next(model.parameters())))  # run once

# model = prune(model)

old_img_b = 4
old_img_h = 640
old_img_w = 640

img = cv2.imread('test.png')
img = img[...,::-1]
img = cv2.resize(img, (640,640))

img = torch.from_numpy(img).to(device)
img = img.unsqueeze(0)
img = torch.moveaxis(img, -1, 1)
img = img.float()
img /= 255.0

for i in range(3):
    model(img, augment=True)[0]

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


In [100]:
def centeroid(a):
    length = a.shape[0]
    sum_x = np.sum(a[:, 0])
    sum_y = np.sum(a[:, 1])
    return int(sum_x/length), int(sum_y/length)

def cluster(centers):
    dist = np.array([np.linalg.norm(centers[0]-center) for center in centers])
    clusters = [0] * len(centers)
    if len(centers) > 1:
        if np.max(dist) > 250:
            kmeans = KMeans(n_clusters=2, n_init=5).fit(centers)
            clusters = list(kmeans.labels_)

        if 1 in clusters:
            ind0 = clusters.index(0)
            ind1 = clusters.index(1)
            clusters = np.array(clusters)
            if centers[ind0, 0] > centers[ind1, 0]:
                clusters = 1 - clusters

    return np.array(clusters)

In [103]:
colors = [[255,0,0], [0,0,255]]

cap = cv2.VideoCapture(0, cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
width = 1920
height = 1080
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)    
while(True):
      
    ret, img = cap.read()
    
    img = cv2.flip(img, 1)
    img0 = img.copy()
    img = img[...,::-1]
    img = cv2.resize(img, (640,640))

    img = torch.from_numpy(img).to(device)
    img = img.unsqueeze(0)
    img = torch.moveaxis(img, -1, 1)
    img = img.float()
    img /= 255.0
    
    with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
        pred = model(img, False)[0]

    conf_thres = 0.1
    iou_thres = 0.01
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes=0, agnostic=True)
    
    for i, det in enumerate(pred):
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
            
#             det_np = det.clone().cpu().numpy()
#             centers = np.array([det_np[:,0] + (det_np[:,2]-det_np[:,0]) // 2, 
#                                 det_np[:,1] + (det_np[:,3]-det_np[:,1]) // 2])
#             centers = np.transpose(centers)
#             clusters = cluster(centers.astype(np.float32))
            
#             if len(clusters) > 0:
#                 centeroidA = centeroid(centers[clusters==0])
#                 if 1 in clusters:
#                     centeroidB = centeroid(centers[clusters==1])
                
#                 cv2.circle(img0, centeroidA, 5, (0, 0, 255), 5)
#                 # cv2.circle(frame, centeroidA, 5, (0, 0, 255), 5)
#                     # if clusters[i] == 1:
#                 if 1 in clusters:
#                     cv2.circle(img0, centeroidB, 5, (255, 0, 0), 5)

            
#             ct = 0
            for *xyxy, conf, cls in reversed(det):
#                 color = colors[clusters[ct]]
                plot_one_box(xyxy, img0, label=None, color=color, line_thickness=1)
#                 ct += 1
    
    cv2.imshow('frame', img0)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [66]:
centers

array([[       1737,        1080],
       [       1740,        1080],
       [       1738,        1080],
       [       1737,        1080],
       [       1738,        1080],
       [       1737,        1080],
       [       1738,        1080],
       [       1737,        1080],
       [       1737,        1080]], dtype=float32)

In [60]:
centeroidA

(183, 1080)